In [17]:
# Load modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.exceptions import FitFailedWarning
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
import seaborn as sns
from Modeling.models.pen_cox import do_lasso_gcv
from sksurv.preprocessing import OneHotEncoder
import re
from sklearn.preprocessing import LabelBinarizer
from sklearn_pandas import DataFrameMapper 
from sklearn.pipeline import make_pipeline
from source.preprocessing import DataLoader

In [3]:
### load data
dl = DataLoader("C:/Users/laeti/OneDrive/Dokumente/LMU/StatisticalConsulting")
dl.load_all_data()

In [4]:
cohorts_exprs_dict = dl.exprs_data
cohorts_pData_dict = dl.pdata_original
intersection_exprs_df = pd.DataFrame(dl.intersection_data['exprs_intersect.csv'])
merged_pData_imputed = pd.DataFrame(dl.merged_pdata_imputed['merged_imputed_pData.csv'])

# 1 Cox-Models

In [59]:
# prepare data for cox
def extract_cohort_name(text):
    regex = r"^([^.]+)\."
    match = re.search(regex, text)
    if match:
        return match.group(1)
    return None

cohort_col = [extract_cohort_name(cor) for cor in merged_pData_imputed.index.tolist()]


merged_pData_imputed
pData_temp = merged_pData_imputed.loc[:, ~merged_pData_imputed.columns.isin(['GLEASON_SCORE_1', 'GLEASON_SCORE_2', 'BCR_STATUS',
                                                                             'MONTH_TO_BCR', 'MONTH_TO_CEP', 'CEP_STATUS'])]
sc_pData = StandardScaler()

#mapper = DataFrameMapper(
#  [(list(set(pData_temp.describe().columns)), StandardScaler())] +
#  [(list(set(pData_temp.columns) - set(pData_temp.describe().columns)), OneHotEncoder())]
#)

pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
pData_temp['PATH_T_STAGE'] = pData_temp['PATH_T_STAGE'].astype('category')
#pData_temp['COHORT'] = cohort_col
#pData_temp['COHORT'] = pData_temp['COHORT'].astype('category')
#X = mapper.fit_transform(pData_temp)

pData_temp_cont = pd.DataFrame(sc_pData.fit_transform(pData_temp[list(set(pData_temp.describe().columns))]), 
                             columns=list(set(pData_temp.describe().columns)), index= pData_temp.index)

pData_temp_cat = OneHotEncoder().fit_transform(pData_temp[(list(set(pData_temp.columns) - set(pData_temp.describe().columns)))])
X = pd.concat([pData_temp_cont, pData_temp_cat, intersection_exprs_df], axis = 1)
#print(pData_temp['COHORT'].info())

dtype = [('event', bool), ('time', float)]

y = np.array([(bool(e), t) for e, t in 
              zip(merged_pData_imputed['BCR_STATUS'], merged_pData_imputed['MONTH_TO_BCR'])], dtype = dtype)




C:\Users\laeti\AppData\Local\Temp\ipykernel_340\4293862249.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['TISSUE'] = pData_temp['TISSUE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_340\4293862249.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pData_temp['CLIN_T_STAGE'] = pData_temp['CLIN_T_STAGE'].astype('category')
C:\Users\laeti\AppData\Local\Temp\ipykernel_340\4293862249.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [60]:
np.mean(X['GLEASON_SCORE'])
X

,PRE_OPERATIVE_PSA,GLEASON_SCORE,AGE,TISSUE=Fresh_frozen,TISSUE=Snap_frozen,PATH_T_STAGE=T1C,PATH_T_STAGE=T2,PATH_T_STAGE=T2A,PATH_T_STAGE=T2B,PATH_T_STAGE=T2C,...,ENSG00000277972,ENSG00000278053,ENSG00000278195,ENSG00000278259,ENSG00000278311,ENSG00000278318,ENSG00000278505,ENSG00000278535,ENSG00000278540,ENSG00000282608
Atlanta_2014_Long.PT081,-0.269899,-0.168861,1.890365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.100000,1.874472,0.412771,-0.201057,-0.023212,1.346444,-0.100000,-0.459605,1.673088,0.236356
Atlanta_2014_Long.PT127,-0.230944,-1.313127,-0.017302,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.100000,0.464558,1.473907,-0.201057,-1.225928,0.861876,-0.100000,-0.459605,1.066850,1.450024
Atlanta_2014_Long.PT168,-0.333250,-1.313127,0.207130,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.100000,0.297391,-0.500212,-0.201057,-1.225928,0.183742,-0.100000,-0.459605,1.766876,-1.059580
Atlanta_2014_Long.PT184,-0.207335,-0.168861,-0.353949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.100000,-0.102693,1.919342,-0.201057,-1.225928,0.086797,-0.100000,-0.459605,-0.051815,0.795535
Atlanta_2014_Long.PT199,-0.269899,-0.168861,-0.466165,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.100000,1.265752,-0.500212,-0.201057,1.013715,-0.187605,-0.100000,-0.459605,0.601972,-1.059580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Stockholm_2016_Ross_Adams.STKHLM8462,-0.278162,-0.168861,-0.313715,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.259790,-0.785377,-0.393627,-0.087867,0.629437,1.177046,-1.055679,0.131266,-0.901919,-0.153249
Stockholm_2016_Ross_Adams.STKHLM8659,-0.152247,-0.168861,-1.067641,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.770326,-1.195006,-2.131437,-0.020858,-0.426975,0.448978,0.720650,-0.539557,0.044055,-0.225257
Stockholm_2016_Ross_Adams.STKHLM9157,-0.132573,-0.168861,0.348379,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.838754,-0.007334,-0.356743,0.636358,-0.564288,1.464345,-2.986711,0.773578,0.778639,-0.771943
Stockholm_2016_Ross_Adams.STKHLM9161,-0.207335,-0.168861,-0.751834,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.816947,-0.640533,-0.966573,1.049117,-0.522911,0.139349,-1.012419,0.657811,1.829945,-0.561905


## 1.1 Penalized Cox-Models (integrated feature selection)

### 1.1.1 Pen. Cox model with cohort as covariate (intersection of genes + pData)

In [57]:
gcv_result = do_lasso_gcv('intersect_cohorts', X, y, n_splits = 5, l1_ratio= 1.0, alpha_min_ratio = 0.2)

In [70]:
gcv_result
pd.DataFrame([gcv_result]).to_csv("pen_cox_exprs_pData_wc.csv")

### 1.1.2 Penalized Cox-Model wo. cohort info

In [61]:
gcv_result_woc = do_lasso_gcv('intersect_cohorts', X, y, n_splits = 5, l1_ratio= 1.0, alpha_min_ratio = 0.2)

In [69]:
gcv_result_woc
pd.DataFrame([gcv_result_woc]).to_csv("pen_cox_pData_exprs_woc.csv")

## 1.2 Cox-Model using with pre-selected features


# 2 Survival forest

## 2.1 Survival forest (integrated feauture selection) 

## 2.2 Survial forest using pre-selected features